In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date,datetime,timedelta
hoje =  pd.to_datetime(date.today() + timedelta(days=25)) 
final_do_mes = pd.to_datetime(datetime(hoje.year,hoje.month+1,1) - timedelta(seconds=1))

Current working directory was changed to: C:\Users\Vinicius\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files,force_download=False)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelacesso já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado no dia: Dia: 17/01/23 - Horário: 00:00:00. Portanto não foi baixado novamente.


In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [4]:
cols = ["IdeUsinaOutorga",'NomUsina','CodCegFormatado','DatMonitoramento',"DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado",'DscJustificativaPrevisao']
df_usina = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]
df_usina = df_usina[df_usina.IdcUsinaMonitorada == "Sim"].reset_index(drop=True)

In [5]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [6]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [7]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [8]:
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [9]:
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [10]:
df_usina["classe"] = df_usina.CodCegFormatado.str.slice(4,6)
df_usina["CC_real"] = pd.to_datetime(pd.NA)
df_usina.loc[df_usina.classe == "PH","CC_real"] = df_usina.DatRealizacaoIV
df_usina.loc[df_usina.classe == "CV","CC_real"] = df_usina.DatRealizacaoV
df_usina["ME_real"] = df_usina.DatRealizacaoX
df_usina.loc[df_usina.classe == "CV","ME_real"] = df_usina.DatRealizacaoVI

In [11]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
df_usina[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
982,46565,NaT,NaT,2022-04-17,2022-04-17
1565,49501,2030-08-25,2030-08-25,2030-08-25,2030-08-25
1467,49375,NaT,NaT,2026-01-06,2026-01-06
1187,47347,2031-02-09,2031-02-09,2031-02-09,2031-02-09
223,34328,NaT,NaT,2023-09-21,2023-09-21


In [12]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao

df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


df_usina[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(df_usina[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(df_usina[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(df_usina[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(df_usina[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2026-03-09,2002-12-11,LI,2026-03-09
1,NaT,NaT,NaT,Sem LP,NaT
2,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
3,NaT,NaT,NaT,Não Informado,NaT
4,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
2441,NaT,NaT,NaT,Não Informado,NaT
2442,NaT,NaT,NaT,Não Informado,NaT
2443,NaT,NaT,NaT,Não Informado,NaT
2444,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
1292,47484,Não se Aplica,Não Assinado,2023-01-03 14:13:24.063,Sem Conexão
2166,53868,Não se Aplica,Não Assinado,2023-01-03 18:36:34.467,Sem Conexão
2055,52073,Não se Aplica,Não Assinado,2023-01-05 14:38:12.047,Sem Conexão
1024,46787,Não se Aplica,Válido,2023-01-04 13:43:26.237,OK
1774,50110,Não se Aplica,Não Assinado,2023-01-05 14:26:11.100,Sem Conexão
1391,48915,Não se Aplica,Não Assinado,2023-01-03 11:55:25.120,Sem Conexão
880,44991,Não se Aplica,Válido,2023-01-05 17:18:48.467,OK
881,44992,Não se Aplica,Não Assinado,2023-01-05 17:24:08.627,Sem Conexão
1466,49374,Não se Aplica,Válido,2023-01-05 18:32:01.763,OK
549,38062,Não se Aplica,Não Assinado,2023-01-04 12:12:34.893,Sem Conexão


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1289,47481,Não se Aplica,Válido,Sem Conexão
1310,48574,Não se Aplica,Não Assinado,Sem Conexão
1299,48518,Não se Aplica,Não Assinado,Sem Conexão
947,46512,Não se Aplica,Não Assinado,Sem Conexão
1073,47058,Não se Aplica,Não Assinado,Sem Conexão
1335,48599,Não se Aplica,Não Assinado,Sem Conexão
1128,47251,Não se Aplica,Não Assinado,Sem Conexão
1599,49634,NaN,NaN,Não informado
570,38112,Não se Aplica,Válido,OK
781,44436,Não se Aplica,Não Assinado,Sem Conexão


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
211,34286,Fora do ACR,NaT,Nenhum
2054,52026,Fora do ACR,NaT,Nenhum
1897,50945,Fora do ACR,NaT,Nenhum
2410,57326,Fora do ACR,NaT,Nenhum
2387,56431,Fora do ACR,NaT,Nenhum
1233,47402,Fora do ACR,NaT,Nenhum
1401,49054,Fora do ACR,NaT,Nenhum
2422,70539,Fora do ACR,NaT,Nenhum
68,30666,Fora do ACR,NaT,Nenhum
1694,49861,Fora do ACR,NaT,Nenhum


In [13]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

In [14]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo
1310,48574,Não Iniciada,Nenhum,LO,Sem Uso,3
625,38393,Não Iniciada,ACL,LI,OK,1
2404,57211,Não Iniciada,Nenhum,LO,Sem Uso,3
476,37748,Não Iniciada,ACR,LP,Sem Uso,1
2438,70555,Não Iniciada,Nenhum,Não Informado,Não informado,7


In [15]:
dict_marcos_homologar = {
    'DatInicioObraRealizado' : 'IO_real',
    'DatConcretagemRealizado' : 'CC_real',
    'DatMontagemRealizado' : 'ME_real',
    'DatEnchimentoRealizado' : 'Ench_Real',
    'DatSisTransmissaoRealizado' : 'IOTrans_Real',
    'DatCanteiroObraRealizado' : 'canteiroReal',
    'DatComissionamentoUGRealizado' : 'Comiss_Real',
    'DatConclusaoSisTransRealizado' : 'Conc_Trans_REAL',
    'DatDesvioRioRealizado' : 'DesvRio_real',
    'DatConclusaoTorresRealizado' : 'ME_Real_conc_eol'}
    
df_usina['Homologar_Marcos'] = False
df_usina['Dsc_Marcos_a_Homologar'] = ''


for marco_monitoramento, marco_rapeel in dict_marcos_homologar.items():
    mask = (df_usina[marco_monitoramento].isna()) & (df_usina[marco_rapeel].notna())
    df_usina['Homologar_Marcos'] |= mask
    df_usina.loc[mask,'Dsc_Marcos_a_Homologar'] += f'{marco_monitoramento}, '

df_usina.Dsc_Marcos_a_Homologar = df_usina.Dsc_Marcos_a_Homologar.str.slice(0,-2)

In [16]:
"""df_usina['Homologar_Marcos'] =  np.select(
    [((df_usina['DatInicioObraRealizado'].isna()) & (df_usina['IO_real'].notna())) |
     ((df_usina['DatConcretagemRealizado'].isna()) & (df_usina['CC_real'].notna())) | 
     ((df_usina['DatMontagemRealizado'].isna()) & (df_usina['ME_real'].notna())) | 
     ((df_usina['DatEnchimentoRealizado'].isna()) & (df_usina['Ench_Real'].notna())) | 
     ((df_usina['DatSisTransmissaoRealizado'].isna()) & (df_usina['IOTrans_Real'].notna())) |
      ((df_usina['DatCanteiroObraRealizado'].isna()) & (df_usina['canteiroReal'].notna())) |
     ((df_usina['DatComissionamentoUGRealizado'].isna()) & (df_usina['Comiss_Real'].notna())) | 
     ((df_usina['DatConclusaoSisTransRealizado'].isna()) & (df_usina['Conc_Trans_REAL'].notna())) | 
     ((df_usina['DatDesvioRioRealizado'].isna()) & (df_usina['DesvRio_real'].notna())) |
     ((df_usina['DatConclusaoTorresRealizado'].isna()) & (df_usina['ME_Real_conc_eol'].notna()))],

    [True],
    default= False
)
"""


df_usina['Revisar_IO'] =  np.select(
 [(df_usina.DatPrevisaoIniciobra < (hoje + pd.Timedelta(15,"D"))) |
 (df_usina.prev_IO > df_usina.DatPrevisaoIniciobra)],
 [True],
 default = False)

In [17]:
usinas_selecionadas = read_file("./usinas_selecionadas.txt").split("\n")
usinas_selecionadas = [int(num) for num in usinas_selecionadas]
df_usina["Usina_Selecionada"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(usinas_selecionadas),'Usina_Selecionada'] = True

df_usina['Paralisada'] = False
df_usina.loc[df_usina.IdcObraParalisada=="Sim",'Paralisada'] = True

df_usina['Sem_Previsao'] = False
df_usina.loc[df_usina.IdcSemPrevisao=="Sim",'Sem_Previsao'] = True

# Informações de UGs

## Carrega banco de dados

In [18]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga",'NumUgUsina',"DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada",'DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})

In [19]:
# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

In [20]:
list_cols_used =   ['IdeUsinaOutorga','IdcObraParalisada','IdcSemPrevisao','criterio_novo','DscJustificativaPrevisao']
df_ug = pd.merge(df_usina[list_cols_used],monitoramentoug,on="IdeUsinaOutorga",how='left')
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left')
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna()].reset_index(drop=True)

In [21]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

In [22]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento','DatInicioObraOutorgado','DatPrevisaoIniciobra']:
    if col in calculo_previsao.columns:
        calculo_previsao.drop([col],inplace=True,axis=1)

In [23]:
df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")

##  Cria colunas com definições e regras

In [24]:
################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)


################################ regranovapmo ################################

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')

df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Dummy"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA


In [25]:
serie_usinas_previsao_OC_passado = df_ug[(df_ug.Previsao_OC < final_do_mes) | (df_ug.Previsao_OC.isna())].IdeUsinaOutorga.drop_duplicates()
serie_usinas_previsao_em_teste = df_ug[df_ug.criterionovopmo.isin([0.2,0.3])].IdeUsinaOutorga.drop_duplicates()

serie_flagOPTeste30dias = df_ug[df_ug.flagOPTeste30dias == 1].IdeUsinaOutorga.drop_duplicates()
df_usina["flagOPTeste30dias"] = False

df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

In [26]:
df_usina["Prev_OC_passado"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_OC_passado),'Prev_OC_passado'] = True

df_usina["Em_teste"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_em_teste),'Em_teste'] = True

df_usina["Sem_Monitoramento"] = False
df_usina.loc[df_usina.DatMonitoramento < (hoje - pd.Timedelta(4*30, unit="D")),'Sem_Monitoramento'] = True

In [27]:
df_usina["Manual"] = False
df_usina.loc[(df_usina.Paralisada | df_usina.Sem_Previsao | 
    df_usina.Usina_Selecionada| df_usina.Prev_OC_passado | df_usina.Em_teste | 
    (df_usina.DscJustificativaPrevisao == 'Situação das obras de conexão e linha de transmissão associada.')),
    'Manual'] = True

In [28]:
df_usina['Caso_I'] = df_usina['Caso_II_a'] = df_usina['Caso_II_b'] = df_usina['Caso_III'] = False
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']

In [29]:
df_usina['Caso_I'] = (~df_usina.Manual) & (~df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos) 

df_usina['Caso_II_a'] = (~df_usina.Manual) & (df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos)

df_usina['Caso_II_b'] = (~df_usina.Manual)  & ((df_usina.Homologar_Marcos) | (df_usina.Homologar_Marcos & df_usina.Revisar_IO))

df_usina['Caso_III'] = (df_usina.Manual & (df_usina.Sem_Monitoramento | df_usina.Homologar_Marcos | df_usina.Revisar_IO)) | df_usina.Em_teste

## Análise de casos BIU

In [30]:
list_criterios_BIU = ['Manual', 'Revisar_IO','Homologar_Marcos', 'Em_teste','Sem_Monitoramento']
casos = df_usina[list_criterios_BIU +  list_casos].value_counts().reset_index().sort_values(by='Caso_III')
casos

,Manual,Revisar_IO,Homologar_Marcos,Em_teste,Sem_Monitoramento,Caso_I,Caso_II_a,Caso_II_b,Caso_III,0
0,False,False,False,False,False,True,False,False,False,2025
1,True,False,False,False,False,False,False,False,False,127
2,False,False,True,False,False,False,False,True,False,82
3,False,True,False,False,False,False,True,False,False,78
6,False,True,True,False,False,False,False,True,False,19
8,False,False,False,False,True,True,False,False,False,1
4,True,False,False,True,False,False,False,False,True,64
5,True,False,False,False,True,False,False,False,True,41
7,True,False,True,False,False,False,False,False,True,8
9,True,False,True,True,False,False,False,False,True,1


In [31]:
mask_casos_selecionados = casos[list_casos].any(axis=1)
casos[mask_casos_selecionados]

,Manual,Revisar_IO,Homologar_Marcos,Em_teste,Sem_Monitoramento,Caso_I,Caso_II_a,Caso_II_b,Caso_III,0
0,False,False,False,False,False,True,False,False,False,2025
2,False,False,True,False,False,False,False,True,False,82
3,False,True,False,False,False,False,True,False,False,78
6,False,True,True,False,False,False,False,True,False,19
8,False,False,False,False,True,True,False,False,False,1
4,True,False,False,True,False,False,False,False,True,64
5,True,False,False,False,True,False,False,False,True,41
7,True,False,True,False,False,False,False,False,True,8
9,True,False,True,True,False,False,False,False,True,1


In [32]:
list_criterios = ['criterio_novo',"criterionovopmo"]

df_usina_criterio = df_ug.loc[df_ug.groupby("IdeUsinaOutorga").criterionovopmo.idxmax()][["IdeUsinaOutorga","FASE",'flagOPTeste30dias'] + list_criterios]

In [33]:
################################ dscjustificativaregranova ################################
list_condicoes = [
    (df_usina_criterio['criterionovopmo'] == 9),
    (df_usina_criterio['criterionovopmo'] == 8),
    (df_usina_criterio['criterionovopmo'] == 0.2),
    (df_usina_criterio['criterionovopmo'] == 0.1) | (df_usina_criterio['criterionovopmo'] == 0.3),
    (df_usina_criterio['criterionovopmo'] == 1),
    df_usina_criterio['criterionovopmo']==2,
    df_usina_criterio['criterionovopmo']==3,
    (df_usina_criterio.criterionovopmo== 4),
    (df_usina_criterio.criterionovopmo== 5),
    (df_usina_criterio.criterionovopmo== 6),
    (df_usina_criterio.criterionovopmo== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_usina_criterio['dscjustificativaregranova'] = np.select(list_condicoes,list_values)



################################ dsccriterionovo ################################
list_condicoes = [
    ((df_usina_criterio['criterio_novo'] == 9)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] != "OT")),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT") & (df_usina_criterio['flagOPTeste30dias'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT")),
    ((df_usina_criterio['criterio_novo'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 2)),
    ((df_usina_criterio['criterio_novo'] == 3)),
    ((df_usina_criterio['criterio_novo'] == 4)),
    ((df_usina_criterio['criterio_novo'] == 5)),
    ((df_usina_criterio['criterio_novo'] == 7)),
    ((df_usina_criterio['criterio_novo'] == 6)),
    ((df_usina_criterio['criterio_novo'] == 8))
]

list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_usina_criterio['dsccriterionovo'] = np.select(list_condicoes,list_values)
list_justificativas = ['dsccriterionovo','dscjustificativaregranova']

In [37]:
df_usina = pd.merge(df_usina,
df_usina_criterio[['IdeUsinaOutorga','criterionovopmo'] + list_justificativas],
on = "IdeUsinaOutorga", how='left')

In [38]:
df_ug = pd.merge(df_usina[['IdeUsinaOutorga','dscjustificativaregranova','dsccriterionovo','DatMonitoramento','DscJustificativaPrevisao']],
                df_ug, on = "IdeUsinaOutorga",how='left')

In [39]:
caso_I = pd.merge(df_usina.loc[df_usina.Caso_I,['IdeUsinaOutorga','dscjustificativaregranova']],
        df_ug[['IdeUsinaOutorga','NumOperacaoUg','Previsao_OC']],
        how='left')

In [132]:
ugs_monitoradas_com_rapeel = monitoramentoug[(monitoramentoug.IdcMonitorada == "Sim") & 
                                (monitoramentoug.IdeUsinaOutorga.isin(ug_rapeel.IdeUsinaOutorga))].copy()

In [133]:
ugs_monitoradas_com_rapeel['Monitoramento'] = True
ug_rapeel['Rapeel'] = True

In [135]:
monitoramentoug[monitoramentoug.IdeUsinaOutorga == 56446]

,DatUGInicioOpComerOutorgado,IdcMonitorada,IdeUsinaOutorga,NumOperacaoUg,DatLiberacaoSFGTeste,DatInicioOpTesteOutorgada,DatLiberOpComerRealizado,Monitoramento


In [134]:
pd.merge(ugs_monitoradas_com_rapeel, ug_rapeel, 'outer', on=list_id_ug)

,DatUGInicioOpComerOutorgado,IdcMonitorada,IdeUsinaOutorga,NumOperacaoUg,DatLiberacaoSFGTeste,DatInicioOpTesteOutorgada,DatLiberOpComerRealizado,Monitoramento,DthEnvio,DatPrevistaComercial,Rapeel
0,2003-10-31,Sim,601,1,NaT,2003-09-30,NaT,True,2023-01-03 19:22:43.140,2027-05-01,True
1,2025-11-18,Sim,27165,8,NaT,NaT,NaT,True,2022-12-05 15:26:51.627,2024-04-01,True
2,2022-10-10,Sim,27917,1,NaT,2022-10-03,NaT,True,2023-01-05 16:00:39.000,2023-04-28,True
3,2024-10-07,Sim,27929,1,NaT,NaT,NaT,True,2023-01-03 15:33:42.707,2023-02-23,True
4,2016-02-01,Sim,28194,1,NaT,2015-11-01,NaT,True,2022-07-05 18:15:50.260,2025-03-01,True
...,...,...,...,...,...,...,...,...,...,...,...
86891,NaT,NaN,56446,12,NaT,NaT,NaT,NaN,2023-01-02 16:16:36.320,2026-10-09,True
86892,NaT,NaN,56446,13,NaT,NaT,NaT,NaN,2023-01-02 16:16:36.320,2026-10-09,True
86893,NaT,NaN,56446,14,NaT,NaT,NaT,NaN,2023-01-02 16:16:36.320,2026-10-09,True
86894,NaT,NaN,56446,15,NaT,NaT,NaT,NaN,2023-01-02 16:16:36.320,2026-10-09,True


In [82]:
count_ug_monitoramento = monitoramentoug.groupby("IdeUsinaOutorga")[['NumOperacaoUg']].nunique().reset_index()
count_ug_rapeel = ug_rapeel.groupby("IdeUsinaOutorga")[['NumOperacaoUg']].nunique().reset_index()
compare_ugs = pd.merge(count_ug_monitoramento,count_ug_rapeel,on='IdeUsinaOutorga')
ugs_qnt_diff = compare_ugs[compare_ugs.NumOperacaoUg_x != compare_ugs.NumOperacaoUg_y].IdeUsinaOutorga
compare_ugs[compare_ugs.NumOperacaoUg_x != compare_ugs.NumOperacaoUg_y]

,IdeUsinaOutorga,NumOperacaoUg_x,NumOperacaoUg_y
0,273,4,2
1,324,2,1
4,742,4,2
8,1997,4,1
9,2092,5,2
...,...,...,...
2692,55739,4,6
2694,55856,4,6
2697,55998,9,13
2701,56139,4,64


In [102]:
monitoramentoug['Monitoramento'] = True
ug_rapeel['Rapeel'] = True
compare_ugs = pd.merge(monitoramentoug,
ug_rapeel, on= list_id_ug, how='outer')

In [103]:
compare_ugs.loc[compare_ugs.Monitoramento.isna(),'Monitoramento'] = False
compare_ugs.loc[compare_ugs.Rapeel.isna(),'Rapeel'] = False

In [104]:
ugs_qnt_diff = compare_ugs[compare_ugs.Monitoramento != compare_ugs.Rapeel].IdeUsinaOutorga.drop_duplicates()

In [118]:
df_ugs_diff = compare_ugs[compare_ugs.Monitoramento != compare_ugs.Rapeel][['IdeUsinaOutorga','NumOperacaoUg','Monitoramento','Rapeel']]
df_ugs_diff.head()

,IdeUsinaOutorga,NumOperacaoUg,Monitoramento,Rapeel
3,955,1,True,False
6,3035,1,True,False
7,27130,2,True,False
12,28063,3,True,False
15,28563,1,True,False


In [122]:
df_ugs_diff_monitoramento = pd.merge(df_ugs_diff,monitoramentoug[list_id_ug + ['IdcMonitorada']],how="left",on=list_id_ug)

In [125]:
df_ugs_diff_monitoramento#[df_ugs_diff_monitoramento.IdcMonitorada == "Sim"]

,IdeUsinaOutorga,NumOperacaoUg,Monitoramento,Rapeel,IdcMonitorada
0,955,1,True,False,Não
1,3035,1,True,False,Não
2,27130,2,True,False,Não
3,28063,3,True,False,Não
4,28563,1,True,False,Não
...,...,...,...,...,...
16757,56446,12,False,True,NaN
16758,56446,13,False,True,NaN
16759,56446,14,False,True,NaN
16760,56446,15,False,True,NaN


In [112]:
compare_ugs[compare_ugs.IdeUsinaOutorga == 27917]

,DatUGInicioOpComerOutorgado,IdcMonitorada,IdeUsinaOutorga,NumOperacaoUg,DatLiberacaoSFGTeste,DatInicioOpTesteOutorgada,DatLiberOpComerRealizado,Monitoramento,DthEnvio,DatPrevistaComercial,Rapeel
9,2022-10-10,Sim,27917,1,NaT,2022-10-03,NaT,True,2023-01-05 16:00:39,2023-04-28,True
191,2022-10-10,Sim,27917,2,NaT,2022-10-03,NaT,True,2023-01-05 16:00:39,2023-04-28,True
318,1976-12-31,Não,27917,5,1976-12-04,1976-12-29,1976-12-04,True,NaT,NaT,False
319,1976-12-31,Não,27917,6,1976-12-04,1976-12-29,1976-12-04,True,NaT,NaT,False
365,1976-12-31,Não,27917,3,1976-12-04,1976-12-29,1976-12-04,True,NaT,NaT,False
798,1976-12-31,Não,27917,4,1976-12-04,1976-12-29,1976-12-04,True,NaT,NaT,False


In [114]:
monitoramentoug[
    monitoramentoug.IdeUsinaOutorga.isin(ugs_qnt_diff) & 
   ( monitoramentoug.IdcMonitorada == "Sim")
]

,DatUGInicioOpComerOutorgado,IdcMonitorada,IdeUsinaOutorga,NumOperacaoUg,DatLiberacaoSFGTeste,DatInicioOpTesteOutorgada,DatLiberOpComerRealizado,Monitoramento
9,2022-10-10,Sim,27917,1,NaT,2022-10-03,NaT,True
17,2004-11-30,Sim,28629,1,NaT,2004-10-30,NaT,True
19,2008-08-15,Sim,28786,1,NaT,2008-07-15,NaT,True
20,2004-10-01,Sim,28792,2,NaT,2004-08-01,NaT,True
23,2024-12-24,Sim,29116,2,NaT,NaT,NaT,True
...,...,...,...,...,...,...,...,...
89919,2027-04-08,Sim,70561,2,NaT,NaT,NaT,True
89920,2027-04-08,Sim,70561,5,NaT,NaT,NaT,True
89921,2027-04-08,Sim,70561,8,NaT,NaT,NaT,True
89922,2027-04-08,Sim,70562,1,NaT,NaT,NaT,True


In [67]:
# Encontrar usinas que enviaram rapeel e que estão monitoradas e que tem UGs diferentes

In [71]:
compare_ugs = pd.merge(monitoramentoug[monitoramentoug.Monitoramento],
ug_rapeel, on= list_id_ug, how='right')

In [66]:
compare_ugs[compare_ugs.Monitoramento.isna() & ].IdeUsinaOutorga.drop_duplicates()

38       27243
41       27395
63       28063
66       28116
95       28818
         ...  
85364    56442
85380    56443
85396    56444
85412    56445
85428    56446
Name: IdeUsinaOutorga, Length: 234, dtype: int32

In [55]:
compare_ugs[(compare_ugs.Monitoramento != compare_ugs.Rapeel) & (compare_ugs.IdcMonitorada == "Sim")].IdeUsinaOutorga.drop_duplicates()

Series([], Name: IdeUsinaOutorga, dtype: int32)

In [ ]:
raise

In [ ]:
df_ug[df_ug.IdeUsinaOutorga == 56139].sort_values(by='NumOperacaoUg')

,IdeUsinaOutorga,dscjustificativaregranova,dsccriterionovo,DatMonitoramento,IdcObraParalisada,IdcSemPrevisao_x,criterio_novo,DscJustificativaPrevisao,DatUGInicioOpComerOutorgado_x,IdcMonitorada,NumOperacaoUg,DatLiberacaoSFGTeste_x,DatInicioOpTesteOutorgada,DatLiberOpComerRealizado_x,DthEnvio,DatPrevistaComercial,MdaPotenciaUnitaria,DatLiberacaoSFGTeste_y,DatLiberOpComerRealizado_y,DatUGInicioOpComerOutorgado_y,DatInicioSuprimento,DscComercializacaoEnergia,NomUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,IdcUsinaMonitorada,IdcSemPrevisao_y,Dat_OC_obrigacao,FASE,DataMarcoAtual,ProximaFase,MarcoMedioAtual,MarcoMedioProximo,AtrasoMarcoAtual,AtrasoMarcoProximo,Tipo de geração_x,a_atual,b_atual,Media_Fase_atual,Std_Fase_atual,Media_Atraso_atual,Std_Atraso_atual,Tipo de geração_y,a_proximo,b_proximo,Media_Fase_proximo,Std_Fase_proximo,Media_Atraso_proximo,Std_Atraso_proximo,Previsao_OC_atual,Previsao_OC_proximo,Previsao_OC,IndicadorAtual,IndicadorProximo,Ind_crono_norm,flagOPTeste30dias,criterionovopmo,regranovapmo,Dummy
73168,56139,Analisar justificativa: Revogação da outorga e...,Usina Viabilidade Baixa análise da fiscalização,2022-12-07 23:53:46.350,Sim,Sim,9,Inviabilidade da implantação da usina,2022-04-20,Sim,1,NaT,2022-04-01,NaT,2023-01-05 19:24:44.557,2023-05-01,25050.0,NaT,NaT,2022-04-20,2022-05-01,Ambos,RE TG 100 02 01,2022-04-05,NaT,NaT,Sim,Sim,2022-05-01,IO,2022-04-05,ME_OC,2020-09-24 14:07:03.529,2021-02-26 12:04:42.352,557 days 09:52:56.470588232,740 days 11:55:17.647058824,UTE,0.981831,-55.131861,583 days 09:52:56.470588232,369 days 14:44:21.736808456,454 days 10:50:32.470588,673 days 01:43:52.614425,UTE,1.014563,-86.675597,428 days 11:55:17.647058824,336 days 06:01:50.966782520,465 days 15:16:35.066413,684 days 03:58:24.901055,2023-09-05 03:39:05.874,2024-02-24 14:31:04.291,2024-02-24 14:31:04.291,43.921088,34.393579,0.343936,0,9.0,NaT,2029-02-22
73171,56139,Analisar justificativa: Revogação da outorga e...,Usina Viabilidade Baixa análise da fiscalização,2022-12-07 23:53:46.350,Sim,Sim,9,Inviabilidade da implantação da usina,2022-04-20,Sim,2,NaT,2022-04-01,NaT,2023-01-05 19:24:44.557,2023-05-01,25050.0,NaT,NaT,2022-04-20,2022-05-01,Ambos,RE TG 100 02 01,2022-04-05,NaT,NaT,Sim,Sim,2022-05-01,IO,2022-04-05,ME_OC,2020-09-24 14:07:03.529,2021-02-26 12:04:42.352,557 days 09:52:56.470588232,740 days 11:55:17.647058824,UTE,0.981831,-55.131861,583 days 09:52:56.470588232,369 days 14:44:21.736808456,454 days 10:50:32.470588,673 days 01:43:52.614425,UTE,1.014563,-86.675597,428 days 11:55:17.647058824,336 days 06:01:50.966782520,465 days 15:16:35.066413,684 days 03:58:24.901055,2023-09-05 03:39:05.874,2024-02-24 14:31:04.291,2024-02-24 14:31:04.291,43.921088,34.393579,0.343936,0,9.0,NaT,2029-02-22
73169,56139,Analisar justificativa: Revogação da outorga e...,Usina Viabilidade Baixa análise da fiscalização,2022-12-07 23:53:46.350,Sim,Sim,9,Inviabilidade da implantação da usina,2022-04-30,Sim,3,NaT,2022-04-10,NaT,2023-01-05 19:24:44.557,2023-05-01,25050.0,NaT,NaT,2022-04-30,2022-05-01,Ambos,RE TG 100 02 01,2022-04-05,NaT,NaT,Sim,Sim,2022-05-01,IO,2022-04-05,ME_OC,2020-09-24 14:07:03.529,2021-02-26 12:04:42.352,557 days 09:52:56.470588232,740 days 11:55:17.647058824,UTE,0.981831,-55.131861,583 days 09:52:56.470588232,369 days 14:44:21.736808456,454 days 10:50:32.470588,673 days 01:43:52.614425,UTE,1.014563,-86.675597,428 days 11:55:17.647058824,336 days 06:01:50.966782520,465 days 15:16:35.066413,684 days 03:58:24.901055,2023-09-05 03:39:05.874,2024-02-24 14:31:04.291,2024-02-24 14:31:04.291,43.921088,34.393579,0.343936,0,9.0,NaT,2029-02-22
73170,56139,Analisar justificativa: Revogação da outorga e...,Usina Viabilidade Baixa análise da fiscalização,2022-12-07 23:53:46.350,Sim,Sim,9,Inviabilidade da implantação da usina,2022-04-30,Sim,4,NaT,2022-04-10,NaT,2023-01-05 19:24:44.557,2023-05-01,25050.0,NaT,NaT,2022-04-30,2022-05-01,Ambos,RE TG 100 02 01,2022-04-05,NaT,NaT,Sim,Sim,2022-05-01,IO,2022-04-05,ME_OC,

In [ ]:
dic = {
    'NumOperacaoUg':'NumUgUsina',
    'regranovapmo': 'PrevisaoOC_regra',
    'dscjustificativaregranova' : 'Justificativadaprevisao_new',
    'criterionovopmo' : 'CriterioPrevisao',
    'dsccriterionovo':'DscCriterioPrevisao',
    'Previsao_OC':'CalculoPrevisorOC',
    'Dat_OC_obrigacao':'OC_Obrigacao',
    'DatPrevistaComercial':'PrevisaoOC_rapeel_max', 
    'DscJustificativaPrevisao':'DscJustificativaPrevisaoAtual'
}   



df_ug.rename(columns=dic)[['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','PrevisaoOC_regra','Justificativadaprevisao_new','CriterioPrevisao','DscCriterioPrevisao','CalculoPrevisorOC','OC_Obrigacao','PrevisaoOC_rapeel_max','DscJustificativaPrevisaoAtual','DatMonitoramento','DthEnvio','FASE',]]

KeyError: "['DscJustificativaPrevisaoAtual'] not in index"

In [ ]:
dic = {'DatPrevisaoIniciobra':'prev_IO_SFG',
        'prev_IO':'prev_IO_rapeel'}
df_usina.rename(columns=dic)[['IdeUsinaOutorga',	'DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']]

,IdeUsinaOutorga,DatInicioObraOutorgado,prev_IO_rapeel,prev_IO_SFG,DatMonitoramento,DthEnvio
0,324,2014-02-28,2025-02-01,NaT,2022-10-13 11:01:56.560,2023-01-02 08:34:07.347
1,601,2001-01-30,2026-05-29,NaT,2022-10-13 11:03:48.340,2023-01-03 19:22:43.140
2,659,2002-04-30,2023-02-10,NaT,2022-12-13 17:04:26.787,2023-01-06 09:37:52.703
3,1195,2002-03-31,NaT,NaT,2022-10-13 11:06:18.107,NaT
4,1358,2014-08-30,2023-02-01,NaT,2022-11-10 16:22:08.193,2022-07-05 18:10:51.733
...,...,...,...,...,...,...
2441,70558,NaT,NaT,2026-11-08,2023-01-15 02:15:48.717,NaT
2442,70559,NaT,NaT,2026-11-08,2023-01-15 02:16:29.043,NaT
2443,70560,NaT,NaT,2025-11-08,2023-01-15 02:17:11.490,NaT
2444,70561,NaT,NaT,2026-11-08,2023-01-15 02:17:52.910,NaT


In [ ]:
#df_usina[df_usina.Homologar_Marcos & (~ df_usina.Sem_Monitoramento)][['IdeUsinaOutorga','NomUsina','Homologar_Marcos',"Sem_Monitoramento",'DatMonitoramento','DthEnvio','Dsc_Marcos_a_Homologar']].to_excel("Usinas_sem_fotos.xlsx",index=False)

# Testes

## Junta dados do SAS e Python

In [ ]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})[["IdeUsinaOutorga","NumOperacaoUg",'CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo','CalculoPrevisorOC_mx',"FASE"]]

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True


#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [ ]:
criterios_da_previsao_sas_gb = criterios_da_previsao_sas.groupby("IdeUsinaOutorga")[['CriterioNovoPMO','DscCriterioNovo','DscJustificativaRegranova']].nunique()
usinas_dois_criterios = criterios_da_previsao_sas_gb[criterios_da_previsao_sas_gb.CriterioNovoPMO>1].index

In [ ]:
criterios_da_previsao_sas_usina = criterios_da_previsao_sas[['IdeUsinaOutorga','CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo']].drop_duplicates()

In [ ]:
merge_usina  = pd.merge(df_usina_criterio[['IdeUsinaOutorga','criterionovopmo','dscjustificativaregranova','dsccriterionovo']],
criterios_da_previsao_sas_usina,on=['IdeUsinaOutorga'],suffixes=('_py','_sas'),how='outer')

In [ ]:
criterios_da_previsao_sas_ug = criterios_da_previsao_sas[list_id_ug + ['CalculoPrevisorOC_mx','FASE']]

In [ ]:
merge_ug = pd.merge(df_ug[list_id_ug + ['FASE','Previsao_OC']],criterios_da_previsao_sas_ug,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge_ug['OC_diff'] = merge_ug.Previsao_OC - merge_ug.CalculoPrevisorOC_mx

## Teste previsão

In [ ]:
merge_ug.sort_values(by='OC_diff',ascending=False).head(20)

,IdeUsinaOutorga,NumOperacaoUg,FASE_py,Previsao_OC,CalculoPrevisorOC_mx,FASE_sas,OC_diff
119,30114,1,OUT,2024-08-14 17:57:40.700,2021-12-01,OUT,987 days 17:57:40.700000
138,30414,1,OUT,2025-03-18 10:06:43.885,2022-09-08,OUT,922 days 10:06:43.885000
120,30114,2,OUT,2024-08-28 00:17:11.536,2022-05-19,OUT,832 days 00:17:11.536000
144,30612,1,OUT,2025-03-29 02:30:59.846,2023-01-23,OUT,796 days 02:30:59.846000
118,30114,3,OUT,2024-09-03 15:26:56.954,2022-08-11,OUT,754 days 15:26:56.954000
11618,37731,3,OUT,2024-09-26 20:18:01.005,2023-02-24,OUT,580 days 20:18:01.005000
11619,37731,1,OUT,2024-09-26 20:18:01.005,2023-02-24,OUT,580 days 20:18:01.005000
11617,37731,2,OUT,2024-09-26 20:18:01.005,2023-02-24,OUT,580 days 20:18:01.005000
11616,37730,3,OUT,2024-10-02 04:30:08.986,2023-05-16,OUT,505 days 04:30:08.986000
11615,37730,2,OUT,2024-10-02 04:30:08.986,2023-05-16,OUT,505 days 04:30:08.986000


## Teste critérios

### FASE ok

In [ ]:
merge_ug[merge_ug.FASE_py != merge_ug.FASE_sas]

,IdeUsinaOutorga,NumOperacaoUg,FASE_py,Previsao_OC,CalculoPrevisorOC_mx,FASE_sas,OC_diff


### criterionovopmo OK

In [ ]:
merge_usina[(merge_usina.CriterioNovoPMO != merge_usina.criterionovopmo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
650,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


In [ ]:
merge_usina[(merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))].sample(6)

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
783,44364,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
779,44362,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
570,38084,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
780,44362,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
174,33548,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
778,44362,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...


### dscjustificativaregranova OK

In [ ]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]


,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
650,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


In [ ]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova)]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
100,32102,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
133,32568,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
139,32580,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
174,33548,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
177,33549,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
375,37101,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
566,38081,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...


### dsccriterionovo OK

In [ ]:
merge_usina[(merge_usina.dsccriterionovo != merge_usina.DscCriterioNovo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
6,1928,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
172,33547,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
513,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
650,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
